In [69]:
from bs4 import BeautifulSoup
import requests
import math
from datetime import datetime

headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }

base_url = 'http://www.teatroallascala.org/archivio/risultato.aspx?lang=en-US&uid=bec93984-a124-4f54-9bfa-cb5d7bd6d71d&objecttype=base&pagesize=9&sort=1&page='

In [70]:
listings = []

for page in range(1,128):
    
    if page%10==0:
        print(page)
        
    url     = base_url + str(page)    
    r       = requests.get(url, headers=headers)
    soup    = BeautifulSoup(r.text, "html5lib")
    results = soup.find_all(attrs={'class': 'risultato'})
    
    for result in results:
        work = result.find('a').get('title')
        
        rows = result.find(attrs={'class': 'inner-risultato'}).find_all('strong')
        composer = ''
        for row in rows:
            if 'music by' in row.get_text().lower():
                composer = row.next_sibling        
        
        #First page of results, only contains 9 items
        season_url   = 'http://www.teatroallascala.org' + result.find('a').get('href')
        season_deets = BeautifulSoup(requests.get(season_url, headers=headers).text, "html5lib")                
        season_results = season_deets.find_all(attrs={'class': 'risultato'})
        for season_result in season_results:
            rows = season_result.find(attrs={'class': 'inner-risultato'}).find_all('strong')
            for row in rows:
                if 'show' in row.get_text().lower():
                    date = row.next_sibling.split(' ')
                    date = [x for x in date if x!='']
                    date = date[1] + '/' + date[2] + '/' + date[3]  
                    date = datetime.strptime(date, '%d/%B/%Y').date()
                                        
                    listing = []
                    listing.append(work)
                    listing.append(composer)
                    listing.append(date)
                    listings.append(listing)
#                     print(listing)                    

        #Subsequent pages of results
        sub_pages = season_deets.find(attrs={'class': 'pagination pagination-small'})
        if sub_pages:
            sub_pages = sub_pages.find_all('a')
            for sub_page in sub_pages[:-1]:
                season_url   = 'http://www.teatroallascala.org' + sub_page.get('href')
                season_deets = BeautifulSoup(requests.get(season_url, headers=headers).text, "html5lib")        
                season_results = season_deets.find_all(attrs={'class': 'risultato'})
                for season_result in season_results:
                    rows = season_result.find(attrs={'class': 'inner-risultato'}).find_all('strong')
                    for row in rows:
                        if 'show' in row.get_text().lower():
                            date = row.next_sibling.split(' ')
                            date = [x for x in date if x!='']
                            date = date[1] + '/' + date[2] + '/' + date[3]  
                            date = datetime.strptime(date, '%d/%B/%Y').date()
                            listing = []
                            listing.append(work)
                            listing.append(composer)
                            listing.append(date)
                            listings.append(listing)
#                             print(listing)

# #Saving results into dataframe and folder            
import pandas as pd                     
headers = ['work', 'composer', 'date']
df = pd.DataFrame(listings, columns=headers)
df.to_csv('../../data/processed/teatro_all_scala_listings.csv')

10
20
30
40
50
60
70
80
90
100
110
120


In [71]:
df.shape

(7615, 3)